In [ ]:
import numpy as np
import os
import glob
import librosa
import torch
import torchaudio

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

From https://pytorch.org/audio/0.10.0/pipelines.html#wav2vec-2-0-hubert-representation-learning 

In [ ]:
# Load the model
bundle = torchaudio.pipelines.HUBERT_BASE # or HUBERT_LARGE 

encoder_embed_dim = 768 # HUBERT_BASE
encoder_embed_dim = 1024 # HUBERT_LARGE

model = bundle.get_model()

In [ ]:
for audio_fn in glob.glob("/homes/lm004/commercials/annotated_commercials/*.mp3"):
    y, _ = librosa.core.load(audio_fn, sr=bundle.sample_rate, res_type='kaiser_fast')

    # Extract acoustic embeddings
    embeddings, _ = model.extract_features(torch.unsqueeze(torch.from_numpy(y), dim=0))

    break

## it returns a list of tensors, what should I do with it?